# 1) Données

In [1]:
import pandas as pd
import numpy as np
import csv
import os

# calcul de la position dans le roulement d'un poste en fonction de la semaine
def rFunction(s,p):
    return ((p-s)%24)

# calcul du numéro de semaine correspondant au jour j
def sFunction(j):
    return j//7

# calcul du jour non travaillé pour une semaine donnée d'un opérateur à 80%
# en fonction du jour non travaillé la première semaine
def delta_i(s,d_i):
    return (d_i+s)%5

# operateurs + competences
dataOperateurs = pd.read_csv('./data/operateurs.csv',sep = ";" )

cardO = len(dataOperateurs.axes[0])
colsDataOperateurs = len(dataOperateurs.axes[1])
nomsOperateurs = dataOperateurs.iloc[0:cardO,0:3]
competencesOperateurs = dataOperateurs.iloc[0:cardO,3:(colsDataOperateurs+1)]

# postes (intitulés + rho)
dataPostes = pd.read_csv('./data/postes.csv',sep = ";" )
cardP = len(dataPostes.axes[0])
nomsPostes = dataPostes.iloc[0:cardO,1]
rho = [0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1]

# le nombre de semaines <=> nombre de postes <=> nombre d'opérateurs
#cardS = cardO
cardS = 17
cardJ = 7*cardS
cardR = cardO

# kappa_ip (opérateur possède la compétence pour le poste)
#kappa = pd.read_csv('./data/kappa_ip.csv',sep = ";" )
kappa = [
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,0,0]]

# sigma_pp' (postes sur le même créneau horaire)
#sigma = pd.read_csv('./data/sigma_pp.csv',sep = ";" )
sigma= [
[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,1,0,0,1,0],
[0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1]]

d = [3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

# o_j (jour ouvrés)
o = np.zeros(cardJ)
for j in range(0,len(o)):
    if (j%7 <= 4):
        o[j] = 1

# H_s (jours de chaque semaine)
H = []
for i in range(0,cardS):
    H.append(np.zeros(7))
for i in range(0,cardS):
    for j in range(0,7):
        H[i][j]=(i*7 + j)
        
profils = np.zeros(cardO)
cardProfils = 0
for i in range(0,cardO):
    found = False
    for j in range(0,i):
        if not found:
            diff = False
            if d[i] == d[j]:
                for p in range(0,cardP):
                    if kappa[i][p] != kappa[j][p]:
                        diff = True
            else:
                diff = True
        
            if not diff:
                profil = profils[j]
                found = True
    if found:
        profils[i] = profil
    else:
        profils[i] = cardProfils
        cardProfils = cardProfils + 1
        
        
#print(profils)
#print(cardProfils)
N = np.zeros(cardProfils)
for i in range(cardO):
    N[int(profils[i])] += 1
#print(N)

# delta_ij (est-ce qu'une personne à 80% travaille le jour j)
#delta = pd.read_csv('./data/delta_ij.csv',sep = ";" )
# d[i] représente le jour non travaillé de l'opérateur i la première semaine, d[i]=-1 si i est à temps plein

delta = np.full((cardProfils, cardJ), 1)
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    if d[index] != -1:
        d_i = d[index]
        for s in range(0, cardS):
            d_is = delta_i(s,d_i) #rang du jour non travaillé la semaine s
            j_s = 7*s + d_is      #numéro du jour non travaillé
            delta[profil][j_s] = 0
            
kappaProfils = []
for profil in range(cardProfils):
    found = False
    i = 0
    index = -1
    while not found and i < cardO:
        if profils[i] == profil:
            found = True
            index = i
        i += 1
    kappaProfils.append(kappa[index])
    
creneaux = [0,2,1,2,0,1,3,1,0,2,1,2,0,1,3,1,0,1,2,1,0,3,1,2,0]
cardCreneaux = len(np.unique(creneaux))
alpha = []
for p in range(cardP):
    ligne = []
    for c in range(cardCreneaux):
        if creneaux[p] == c:
            ligne.append(1)
        else:
            ligne.append(0)
    alpha.append(ligne)

#print(nomsOperateurs)
#print(competencesOperateurs)
#print(nomsPostes)
#print(rho)
#print(kappa)
#print(sigma)
#print(delta)
#print(o)
#print(H[1])
#print(kappaProfils)
#print(delta[0])

# 2) Modèle mathématique


## 2.1) Variables

In [2]:
from pulp import *

# x_ijp = 1 si l'opérateur prend en charge le poste p le jour j
x = {} 
for prof in range(0,cardProfils):
    for j in range(0,cardJ): 
        for p in range(0,cardP): 
            x[prof,j,p]=LpVariable(f"x({prof,j,p})", cat=LpBinary) #LpBinary

# y_ir = 1 si l'opérateur i est position r dans le roulement
y = {} 
for prof in range(0,cardProfils):
    for r in range(0,cardR):    
        y[prof,r]=LpVariable(f"y({prof,r})", cat=LpBinary)
        
# z_isp = 1 si l'opérateur est affecté au poste p la semaine s
z = {} 
for prof in range(0,cardProfils):
    for s in range(0,cardS): 
        for p in range(0,cardP): 
            z[prof,s,p]=LpVariable(f"z({prof,s,p})", cat=LpBinary)

# u_is = 1 si l'opérateur i est affecté à un poste différent de celui prévu la semaine s
u = {} 
for p in range(0,cardP):
    for s in range(0,cardS):    
        u[p,s]=LpVariable(f"u({p,s})", cat=LpBinary)

print("nb de variables x_ijp = ", len(x))
print("nb de variables y_ir  = ", len(y))
print("nb de variables z_isp = ", len(z))
print("nb de variables u_ir  = ", len(u))
print()
print("nb total de variables = ", len(x)+len(y)+len(z)+len(u))

nb de variables x_ijp =  14280
nb de variables y_ir  =  120
nb de variables z_isp =  2040
nb de variables u_ir  =  408

nb total de variables =  16848


## 2.2) Création du modèle avec PuLP

In [3]:
prob = LpProblem("ModeleProfils", LpMinimize)

prob += sum(u[p,s] for p in range(0,cardP) for s in range(0,cardS)), "obj"

## 2.2) Contraintes

In [4]:
#contrainte 1 Un profil opérateur ne peut prendre un poste que s'il est compétent et disponible sur un jour ouvré

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        for p in range(0,cardP):
            prob += (x[i,j,p] <= delta[i][j]*kappaProfils[i][p]*o[j], f"C1_{i,j,p}")

#contrainte 2 Un profil opérateur ne peut être affecté à un poste une semaine que s'il en possède la compétence

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += (z[i,s,p] <= kappaProfils[i][p], f"C2_{i,s,p}")

#contrainte 3 Chaque jour, un profil opérateur est affecté à son poste de la semaine (s'il n'est pas rouleur)
for i in range(0,cardProfils):
    for s in range(0,cardS):
        for j in H[s]:
            for p in range(0,cardP):
                prob += x[i,j,p] <= z[i,s,p] + sum((1 - rho[p2])*z[i,s,p2] for p2 in range(0,cardP)), f"C3_{i,j,p}"

#contrainte 4 Chaque jour, un profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for j in range(0,cardJ):
        if o[j] == 1 and delta[i][j] == 1:
            prob += sum(x[i,j,p] for p in range(0,cardP)) == N[i], f"C4_{i,j}"

#contrainte 5 Chaque jour, chaque poste non rouleur est assuré par exactement un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 1:
            prob += sum(x[i,j,p] for i in range(cardProfils)) == 1, f"C5_{j,p}"
            
#contrainte 6 Chaque jour, les postes rouleurs sont occupés par au plus un profil opérateur
for p in range(0,cardP):
    for j in range(0,cardJ):
        if o[j] == 1 and rho[p] == 0:
            prob += sum(x[i,j,p] for i in range(0,cardProfils)) <= 1, f"C6_{j,p}"

#contrainte 7 Chaque semaine, chaque profil opérateur est affecté à autant de poste que son nombre de membres

for i in range(0,cardProfils):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for p in range(0,cardP)) == N[i], f"C7_{i,s}"

#contrainte 8 Chaque semaine, chaque poste est assuré par un profil opérateur

for p in range(0,cardP):
    for s in range(0,cardS):
        prob += sum(z[i,s,p] for i in range(0,cardProfils)) == 1, f"C8_{s,p}"

#contrainte 9 Chaque profil opérateur est affecté à autant de positions dans le roulement que son nombre de membres

for i in range(0,cardProfils):
    prob += sum(y[i,r] for r in range(0,cardR)) == N[i], f"C9_{i}"

#contrainte 10 Chaque position dans le roulement est associée à un profil opérateur

for r in range(0,cardR):
    prob += sum(y[i,r] for i in range(0,cardProfils)) == 1, f"C10_{r}"

#contrainte 11 Un profil opérateur peut permuter de poste si leurs horaires sont compatibles

for i in range(0,cardProfils):
    for s in range(0,cardS):
        for p in range(0,cardP):
            prob += z[i,s,p] <= sum(sigma[p][p2]*y[i,rFunction(s,p2)] for p2 in range(0,cardP)), f"C11_{i,s,p}"

#contrainte 12 Lorsqu'un poste est occupé par un autre profil opérateur que celui prévu initialement,
#son score d'insatisfaction augmente
for s in range(0,cardS):
    for p in range(0,cardP):
        for i in range(cardProfils):
            prob += u[p,s] >= (kappaProfils[i][p]*rho[p]*y[i,rFunction(s,p)] - z[i,s,p]), f"C12_{i,s,p}"
            
#contrainte David anti permutation interdite
for s in range(cardS):
    for i in range(cardProfils):
        for c in range(cardCreneaux):
            prob += sum(alpha[p][c]*z[i,s,p] for p in range(cardP)) == sum(alpha[p][c]*y[i,rFunction(s,p)] for p in range(cardP)), f"C13_{s,i,c}"


## 2.4) Résolution du problème

In [5]:
import time
import cplex
from cplex.callbacks import MIPInfoCallback
log = []

class MyCallback(MIPInfoCallback):

    def __call__(self):
        current = time.time()
        log.append([self.get_incumbent_objective_value(), current - start])
        
prob.writeLP("ModeleProfils.lp")

c = cplex.Cplex()
c.parameters.timelimit.set(9960.0)
c.register_callback(MyCallback)
c.read("ModeleProfils.lp")

#prob.writeLP("Model.lp")
#solver = CPLEX_PY()
#solver.buildSolverModel(prob)
##solver.callSolver(prob)
#prob.solve(solver.solverModel.parameters.timelimit.set(1))

start = time.time()
#prob.solve(CPLEX_PY())
c.solve()
solution = c.solution
print(solution.get_objective_value())
end = time.time()

duree = end - start
print("Temps de résolution : " + str(duree))

print("log")
for i in range(len(log)):
    print(log[i])

#import time
#prob.writeLP("Model.lp")
#start = time.time()
#prob.solve(CPLEX_PY())
#end = time.time()
#duree = end - start
#print("Temps de résolution : " + str(duree))



Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               9960
Legacy callback                                  i
Tried aggregator 2 times.
MIP Presolve eliminated 23987 rows and 5712 columns.
MIP Presolve modified 2448 coefficients.
Aggregator did 17 substitutions.
Reduced MIP has 13986 rows, 11119 columns, and 98925 nonzeros.
Reduced MIP has 11119 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.12 sec. (130.99 ticks)
Found incumbent of value 306.000000 after 0.25 sec. (219.27 ticks)
Probing changed sense of 510 constraints.
Probing time = 0.20 sec. (118.61 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 697 rows and 510 columns.
Reduced MIP has 13289 rows, 10609 columns, and 96375 nonzeros.
Reduced MIP has 10609 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.19 sec. (160.42 ticks)
Probing time = 0.02 sec. (9.67 ticks)
Cli

    506   373        4.5667  1622        9.0000        3.9033   696298   56.63%
*   510+  359                            8.0000        3.9033            51.21%
    549   377        6.9839   967        8.0000        3.9033   696604   51.21%
    585   386        5.4452  1443        8.0000        3.9087   820064   51.14%
    633   387        5.9935  1179        8.0000        3.9087   819488   51.14%
    667   438        5.1518  1234        8.0000        3.9760   932045   50.30%
    708   493        5.6489   686        8.0000        3.9760  1023474   50.30%
    747   469        4.0493  1891        8.0000        3.9760   998569   50.30%
    787   518        7.0000   612        8.0000        3.9872  1075758   50.16%
    839   555        5.3159  1724        8.0000        3.9917  1152699   50.10%
Elapsed time = 154.38 sec. (61678.40 ticks, tree = 2.36 MB, solutions = 16)
    877   566        4.2131  1668        8.0000        3.9917  1173921   50.10%
    923   571        4.9175   818        8.0

   3971   458        5.1518  1705        8.0000        4.4194  5335958   44.76%
   4153   571        5.3964  1367        8.0000        4.4194  5423230   44.76%
   4336   698        cutoff              8.0000        4.4194  5547558   44.76%
   4515   831        4.8188  1948        8.0000        4.4194  5685535   44.76%
   4696   971        6.6231  1085        8.0000        4.4194  5825320   44.76%
   4894  1112        5.7232  1234        8.0000        4.4194  5961972   44.76%
   5069  1254        6.2441  1098        8.0000        4.4194  6086696   44.76%
   5250  1406        5.0061  1602        8.0000        4.4194  6241630   44.76%
   5423  1579        4.5347  2035        8.0000        4.4194  6415187   44.76%
Elapsed time = 497.67 sec. (191558.89 ticks, tree = 37.38 MB, solutions = 16)
   5540  1632        5.5648  1808        8.0000        4.4194  6469995   44.76%
   5667  1699        5.9856  1692        8.0000        4.4194  6555226   44.76%
   5836  1814        4.8917  1858        8

## 2.5) Extraction de la solution

In [6]:
print("objectif =", c.solution.get_objective_value())

objectif = 8.0


In [7]:
#print("objectif =", value(prob.objective))
print("objectif =", c.solution.get_objective_value())
#print("var =", c.solution.get_values(y[0,0]))
for s in range(cardS):
    for p in range(cardP):
        val = c.solution.get_values("u(("+str(p)+",_"+str(s)+"))")
        if val != 0:
            print("u[",p,",",s,"] =", val)

print()
for i in range(cardProfils):
    for r in range(cardR):
        val = c.solution.get_values("y(("+str(i)+",_"+str(r)+"))")
        if val != 0:
            print("y[",i,",",r,"] =", val)
        
print()
for s in range(cardS):
    print()
    for i in range(cardProfils):
        for p in range(cardP):
            val = c.solution.get_values("z(("+str(i)+",_"+str(s)+",_"+str(p)+"))")
            if val != 0:
                print("z[",i,",",s,",",p,"] =", val)

#print(c.variables.get_names())
#solver.solution.get_objective_value()
#print(objval)

objectif = 8.0
u[ 6 , 0 ] = 1.0
u[ 19 , 0 ] = 1.0
u[ 10 , 1 ] = 1.0
u[ 1 , 2 ] = 1.0
u[ 6 , 14 ] = 1.0
u[ 19 , 15 ] = 1.0
u[ 5 , 16 ] = 1.0
u[ 6 , 16 ] = 1.0

y[ 0 , 14 ] = 1.0
y[ 1 , 3 ] = 1.0
y[ 1 , 4 ] = 1.0
y[ 1 , 5 ] = 1.0
y[ 1 , 6 ] = 1.0
y[ 1 , 7 ] = 1.0
y[ 1 , 8 ] = 1.0
y[ 1 , 9 ] = 1.0
y[ 1 , 10 ] = 1.0
y[ 1 , 11 ] = 1.0
y[ 1 , 12 ] = 1.0
y[ 1 , 13 ] = 1.0
y[ 1 , 15 ] = 1.0
y[ 1 , 20 ] = 1.0
y[ 2 , 2 ] = 1.0
y[ 2 , 16 ] = 1.0
y[ 2 , 18 ] = 1.0
y[ 2 , 19 ] = 1.0
y[ 2 , 23 ] = 1.0
y[ 3 , 0 ] = 1.0
y[ 3 , 1 ] = 1.0
y[ 3 , 21 ] = 1.0
y[ 3 , 22 ] = 1.0
y[ 4 , 17 ] = 1.0


z[ 0 , 0 , 14 ] = 1.0
z[ 1 , 0 , 3 ] = 1.0
z[ 1 , 0 , 4 ] = 1.0
z[ 1 , 0 , 5 ] = 1.0
z[ 1 , 0 , 7 ] = 1.0
z[ 1 , 0 , 8 ] = 1.0
z[ 1 , 0 , 9 ] = 1.0
z[ 1 , 0 , 10 ] = 1.0
z[ 1 , 0 , 11 ] = 1.0
z[ 1 , 0 , 12 ] = 1.0
z[ 1 , 0 , 13 ] = 1.0
z[ 1 , 0 , 15 ] = 1.0
z[ 1 , 0 , 20 ] = 1.0
z[ 1 , 0 , 21 ] = 1.0
z[ 2 , 0 , 2 ] = 1.0
z[ 2 , 0 , 16 ] = 1.0
z[ 2 , 0 , 18 ] = 1.0
z[ 2 , 0 , 22 ] = 1.0
z[ 2 , 0 , 23 ] = 1.0
z[ 3 ,

## 2.6) Affichage de la solution

In [8]:
print("uis")
for s in range(0,cardS):
    for p in range(0,cardP):
        if(u[p,s].varValue != 0):
            print("u[",p,",",s,"] =", u[p,s].varValue)

print('Affectation aux roulements')
for i in range(cardProfils):
    for r in range(0,24):
        if(y[i,r].varValue != 0):
            print('y[',i,',',r,'] =', y[i,r].varValue)

print()
print('Affectations hebdomadaires')
for s in range(0,cardS):
    print('semaine',s)
    for i in range(0,cardProfils):
        for p in range(0,cardP):
            if z[i,s,p].varValue==1:
                print('semaine',s,',',i,'affecté au poste',p)
                
print('======================================================')

for p in range(0,cardP):
    print()
    if rho[p] == 1:
        print('poste',p,'non rouleur')
    else:
        print('poste',p,'rouleur')
    for j in range(0,cardJ):
        for i in range(0,cardProfils):
            if (x[i,j,p].varValue == 1):
                print(i,'occupe le poste',p,'le jour',j)
                
print('======================================================')

                
for i in range(0,cardO):
    print()
    print('planning de l\'opérateur',i)
    for j in range(0,cardJ):
        for p in range(0,cardP):
            if (x[i,j,p].varValue == 1):
                print('poste',p,'le jour',j)

uis
u[ 0 , 0 ] = None
u[ 1 , 0 ] = None
u[ 2 , 0 ] = None
u[ 3 , 0 ] = None
u[ 4 , 0 ] = None
u[ 5 , 0 ] = None
u[ 6 , 0 ] = None
u[ 7 , 0 ] = None
u[ 8 , 0 ] = None
u[ 9 , 0 ] = None
u[ 10 , 0 ] = None
u[ 11 , 0 ] = None
u[ 12 , 0 ] = None
u[ 13 , 0 ] = None
u[ 14 , 0 ] = None
u[ 15 , 0 ] = None
u[ 16 , 0 ] = None
u[ 17 , 0 ] = None
u[ 18 , 0 ] = None
u[ 19 , 0 ] = None
u[ 20 , 0 ] = None
u[ 21 , 0 ] = None
u[ 22 , 0 ] = None
u[ 23 , 0 ] = None
u[ 0 , 1 ] = None
u[ 1 , 1 ] = None
u[ 2 , 1 ] = None
u[ 3 , 1 ] = None
u[ 4 , 1 ] = None
u[ 5 , 1 ] = None
u[ 6 , 1 ] = None
u[ 7 , 1 ] = None
u[ 8 , 1 ] = None
u[ 9 , 1 ] = None
u[ 10 , 1 ] = None
u[ 11 , 1 ] = None
u[ 12 , 1 ] = None
u[ 13 , 1 ] = None
u[ 14 , 1 ] = None
u[ 15 , 1 ] = None
u[ 16 , 1 ] = None
u[ 17 , 1 ] = None
u[ 18 , 1 ] = None
u[ 19 , 1 ] = None
u[ 20 , 1 ] = None
u[ 21 , 1 ] = None
u[ 22 , 1 ] = None
u[ 23 , 1 ] = None
u[ 0 , 2 ] = None
u[ 1 , 2 ] = None
u[ 2 , 2 ] = None
u[ 3 , 2 ] = None
u[ 4 , 2 ] = None
u[ 5 , 2 ] = N

KeyError: (5, 0, 0)

# Extraction des données au format CSV

In [ ]:
import csv
with open('res.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    jours = [""]
    for s in range(0,cardS):
        jours.extend(["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"])
    writer.writerow(jours)
    
    dates = [""]
    for j in range(0,cardJ):
        dates.append("jour "+str(j))
    writer.writerow(dates)
    
    for i in range(0,cardO):
        planningPerso = [nomsOperateurs.iloc[i,1] + " " + nomsOperateurs.iloc[i,2]]
        for j in range(0,cardJ):
            found = False
            for p in range(0,cardP):
                if x[i,j,p].varValue == 1:
                    found = True
                    planningPerso.append(nomsPostes.iloc[p])
            if not found:
                if o[j] == 0:
                    planningPerso.append("")
                else:
                    planningPerso.append("RA")
        writer.writerow(planningPerso)